In [22]:
import pandas as pd
import re

In [119]:
df_parc = pd.read_csv('./../../../Data/parc_error_analysis.csv', sep='\t', index_col=0)
df_polnear = pd.read_csv('./../../../Data/polnear_error_analysis.csv', sep='\t', index_col=0)
df_polnear.drop(['filename_y', 'doc_token_number_y'], axis=1, inplace=True)
df_polnear.rename(columns={'filename_x':'filename'}, inplace=True)

In [120]:
for column_name in ['gold_attribution', 'pred_attribution']:
    cleaned_atts = []
    for element in df_parc[column_name]:
        if '-NE-' in element:
            element = re.sub("[B,I]-.+-NE-.+", "", element)
            #element = element.replace('.-...-NE-..', '')
        cleaned_atts.append(element)

    df_parc[column_name] = cleaned_atts
    df_parc[column_name] = df_parc[column_name].apply(lambda row: row.strip('_ '))

In [102]:
def unmatched_contents(df):
    filenames = set(df['filename'])

    count_unmatched = 0
    count_total = 0
    for filename in filenames:
        df_per_file = df[df['filename'] == filename]
        att_label_set = set(df_per_file['pred_attribution'])

        content_ids = set()
        source_ids = set()
        for att_label in att_label_set:
            att_split = att_label.split('-')
                if len(att_split) < 4:
                    continue
                if att_split[1] == 'CONTENT':
                    content_ids.add(att_split[3])
                elif att_split[1] == 'SOURCE':
                    source_ids.add(att_split[3])

        count_unmatched += len(content_ids - source_ids)
        count_total += len(content_ids)
        if len(content_ids - source_ids) != 0:
            print(filename)

    return (count_unmatched, count_total)

In [103]:
print(unmatched_contents(df_parc))
print(unmatched_contents(df_polnear))

wsj_2450.tsv
wsj_2407.tsv
wsj_2425.tsv
wsj_2412.tsv
wsj_2418.tsv
wsj_2417.tsv
wsj_2413.tsv
(9, 487)
wash-post_2016-09-18_e-u-leaders-weigh-plans-for-grea.tsv
politico_2016-09-25_poll-clinton-and-trump-in-dead-h.tsv
usa-today_2016-09-08_candidates-vie-for-vets-support-.tsv
politico_2016-09-23_partisan-fireworks-over-clinton-.tsv
politico_2016-09-30_flight-attendants-union-endorses.tsv
(5, 854)


In [104]:
df_parc[df_parc['gold_attribution'] != '']

,filename,token,POS,doc_token_number,sentence_number,sentence_token_number,ne_info,relevant_ne,cue_label,candidate_source_label,gold_attribution,pred_attribution
40,wsj_2400.tsv,The,DT,41,3,1,O,0,0,O,B-CONTENT-PD-0,B-CONTENT-XX-0
41,wsj_2400.tsv,trade,NN,42,3,2,O,0,0,B,I-CONTENT-PD-0,I-CONTENT-XX-0
42,wsj_2400.tsv,gap,NN,43,3,3,O,0,0,B,I-CONTENT-PD-0,I-CONTENT-XX-0
59,wsj_2400.tsv,according,VBG,60,3,20,O,0,0,O,B-CUE-PD-0,
60,wsj_2400.tsv,to,IN,61,3,21,O,0,1,O,I-CUE-PD-0,
61,wsj_2400.tsv,a,DT,62,3,22,O,0,0,O,B-SOURCE-PD-0,
62,wsj_2400.tsv,survey,NN,63,3,23,O,0,0,B,I-SOURCE-PD-0,B-SOURCE-XX-0
63,wsj_2400.tsv,by,IN,64,3,24,O,0,0,O,I-SOURCE-PD-0,
64,wsj_2400.tsv,MMS,NNP,65,3,25,B-ORG,1,0,B,I-SOURCE-PD-0,I-SOURCE-XX-0
65,wsj_2400.tsv,International,NNP,66,3,26,E-ORG,1,0,I,I-SOURCE-PD-0,I-SOURCE-XX-0


In [129]:
def count_candidate_sources(df):
    filenames = set(df['filename'])

    missed_count = 0
    partial_count = 0
    total_count = 0

    for filename in filenames:
        cand_source_dict = dict()

        for index, row in df.iterrows():
            att_split = row['gold_attribution'].split('-')
            if len(att_split) != 4:
                continue
            if att_split[1] == 'SOURCE':
                att_id = att_split[3]
                if att_id not in cand_source_dict:
                    cand_source_dict[att_id] = set()
                cand_source_dict[att_id].add(row['candidate_source_label'])


        for source_id, cand_labels in cand_source_dict.items():
            if 'O' in cand_labels and len(cand_labels) == 1:
                missed_count += 1
            elif 'O' in cand_labels and len(cand_labels) > 1:
                partial_count += 1
            elif 'O' not in cand_labels:
                total_count += 1

    print(missed_count, partial_count, total_count)

In [130]:
count_candidate_sources(df_parc)

0 2145 110


In [ ]:
count_candidate_sources(df_polnear)